In [1]:
import numpy as np 
import astropy.constants as c
import os
import csv
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import curve_fit
import math
import gc
#list all the constants used
G=c.G.value
M_sun=c.M_sun.value
M_earth=c.M_earth.value
R_sun=c.R_sun.value
R_earth=c.R_earth.value
hour=3600 #s
day=3600*24
e=0.1
t_LC=1765.5 #s
t_obs=17*90

def delta(Rp, Rs):
    delta = (Rp*R_earth / (Rs*R_sun))**2*1e6
    return delta

def SNR(delta,n,sigma_s):
    SNR=delta*n**0.5/sigma_s
    return SNR

def sigma_s(t, sigma_LC, cdppindex): #t/hour
    t_LC=1765.5/hour 

    return sigma_LC*(t/t_LC)**cdppindex

def t_dur(P,Rs,e,a): #hour
    t_dur = P*day*Rs*R_sun*(1-e**2)/(np.pi*a)/hour
    return t_dur

def a(Ms,P):
    a=((G*Ms*M_sun*(P*day)**2)/(4*np.pi**2))**(1/3)
    return a 

def n(t_obs,P): # the unite of t_obs and P is day
    n=t_obs/P
    return n

def f_n(t_obs,P):
    if t_obs <=2*P:
        return 0
    if t_obs> 2*P and t_obs< 3*P:
        return (t_obs/P-2)
    if t_obs>=3*P:
        return 1
    
def f_eff(SNR):
    if SNR <=6:
        return 0
    if SNR >6 and SNR<=12:
        return (SNR-6)/6
    if SNR>12:
        return 1
    
def f_geo(Rp,Rs,a,e):
    f_geo = (Rp*R_earth+Rs*R_sun)/(a*(1-e**2))
    return f_geo

In [5]:
stellar_table=pd.read_csv('data/kepler_stellar_q17_add_dispersions_reduction_reduction_cool_2020.csv',sep=',')
koi_table=pd.read_csv('data/kepler_koi_q17_add_dispersions_reduction_reduction_2sigma_cool.csv',sep=',')

def N_s(Rp,P,i):
    os.system('mkdir result/N_cool_2020/')
    N_s_temp=0
    with open('result/N_cool_2020/'+str(i)+'.csv','w') as f:
        s=['SNR',
           'fn',
           'feff',]
        writer=csv.DictWriter(f,fieldnames=s)
        writer.writeheader()        
        for i in range(0,len(stellar_table)):
            a_temp=a(stellar_table['mass'][i],P)
            tdur_temp=t_dur(P,stellar_table['rad_b2018'][i],e,a_temp)
            delta_temp=delta(Rp,stellar_table['rad_b2018'][i])
            n_temp=n(stellar_table['t_obs'][i],P)
            x=[3,6,12]
            y=[stellar_table['cdpp3'][i],stellar_table['cdpp6'][i],stellar_table['cdpp12'][i]]
            popt, pcov = curve_fit(sigma_s, x, y)
            sigma_LC=popt[0]
            cdppindex=popt[1]
            sigma_s_temp=sigma_s(tdur_temp,sigma_LC,cdppindex)
            SNR_temp=SNR(delta_temp,n_temp,sigma_s_temp)
            fn_temp=f_n(stellar_table['t_obs'][i],P)
            feff_temp=f_eff(SNR_temp)
            writer.writerow({'SNR':SNR_temp,
                             'fn':fn_temp,
                             'feff':feff_temp,})                             
            N_s_temp+=fn_temp*feff_temp
    return round(N_s_temp)

def N_s2(Rp,P):
#    os.system('mkdir result/N_s/')
    N_s_temp=0
#    with open('result/N_s/'+str(kepid)+'.csv','w') as f:
#        s=['SNR',
#           'fn',
#           'feff',]
#        writer=csv.DictWriter(f,fieldnames=s)
#        writer.writeheader()        
    for i in range(0,len(stellar_table)):
        a_temp=a(stellar_table['mass'][i],P)
        tdur_temp=t_dur(P,stellar_table['rad_b2018'][i],e,a_temp)
        delta_temp=delta(Rp,stellar_table['rad_b2018'][i])
        n_temp=n(stellar_table['t_obs'][i],P)
        x=[3,6,12]
        y=[stellar_table['cdpp3'][i],stellar_table['cdpp6'][i],stellar_table['cdpp12'][i]]
        popt, pcov = curve_fit(sigma_s, x, y)
        sigma_LC=popt[0]
        cdppindex=popt[1]
        sigma_s_temp=sigma_s(tdur_temp,sigma_LC,cdppindex)
        SNR_temp=SNR(delta_temp,n_temp,sigma_s_temp)
        fn_temp=f_n(stellar_table['t_obs'][i],P)
        feff_temp=f_eff(SNR_temp)
#        writer.writerow({'SNR':SNR_temp,
#                         'fn':fn_temp,
#                         'feff':feff_temp,})                             
        N_s_temp+=fn_temp*feff_temp
    return round(N_s_temp)

#######################################3
def f_occurrence_dot(Rp,P,Ms,Rs,e,i):
    N_s_temp=N_s(Rp,P,i)
    a_temp=a(Ms,P)
    fgeo_temp=f_geo(Rp,Rs,a_temp,e)
    f_occurrence_temp=1./(fgeo_temp*N_s_temp)
    return f_occurrence_temp
    
##############################################
def f_occurrence_cell_nofp(Rp_low,Rp_high,P_low,P_high): # the log base of Rp and P
    focc=pd.read_csv('result/occurrence_rate_cool_2020_2sigma.csv')
    f_occurrence_everyplanet=focc['occurrence_rate']
    f_id=focc['i']    
    count=0
    for i in range(0,len(f_occurrence_everyplanet)):
        if koi_table['koi_prad'][f_id[i]]>Rp_low and koi_table['koi_prad'][f_id[i]]<=Rp_high and koi_table['koi_period'][f_id[i]]>P_low and koi_table['koi_period'][f_id[i]]<=P_high\
        and koi_table['koi_disposition'][f_id[i]]!='FALSE POSITIVE':
            count+=1
    
    if count>0:
        f_occurrence_temp=0
        for i in range(0,len(f_occurrence_everyplanet)):
            if koi_table['koi_prad'][f_id[i]]>Rp_low and koi_table['koi_prad'][f_id[i]]<=Rp_high and koi_table['koi_period'][f_id[i]]>P_low and koi_table['koi_period'][f_id[i]]<=P_high\
            and koi_table['koi_disposition'][f_id[i]]!='FALSE POSITIVE':
#                N_s_temp=N_s(koi_table['koi_prad'][i],koi_table['koi_period'][i])
#                a_temp=a(koi_table['mass'][i],koi_table['koi_period'][i])
#                fgeo_temp=f_geo(koi_table['koi_prad'][i],koi_table['rad_b2018'][i],a_temp,e)
                if f_occurrence_everyplanet[i]!=np.inf:
                    f_occurrence_temp+=float(f_occurrence_everyplanet[i])
        
        temp=1.0*count/f_occurrence_temp
        if f_occurrence_temp>=1:
            return count,f_occurrence_temp,f_occurrence_temp/count**(0.5)
        else:
            deviation_N_s=temp*f_occurrence_temp*(1-f_occurrence_temp)
            std_N_s=deviation_N_s**0.5
            std_cell=std_N_s/temp
            return count,f_occurrence_temp,std_cell
    
    else:
        base_period=5
        base_radius=2
        Rp_center=base_radius**((math.log(Rp_low,base_radius)+math.log(Rp_high,base_radius))/2)
        P_center=base_period**((math.log(P_low,base_period)+math.log(P_high,base_period))/2)
        N_s_temp=N_s2(Rp_center,P_center)
        Rs_temp=1.0 #unit R_sun G type star as sun
        Ms_temp=1.0 #unit M_sun
        a_temp=a(Ms_temp,P_center)
        e=0.1
        fgeo_temp=f_geo(Rp_center,Rs_temp,a_temp,e)
        if N_s_temp==0:
            f_occurrence_temp=0
        else:
            f_occurrence_temp=1./(fgeo_temp*N_s_temp)
            
        return count,f_occurrence_temp,f_occurrence_temp
    
#####################################################3
def f_occurrence_cell_fp(Rp_low,Rp_high,P_low,P_high): # the log base of Rp and P
    focc=pd.read_csv('result/occurrence_rate_cool_2020_2sigma.csv')
    f_occurrence_everyplanet=focc['occurrence_rate']
    f_id=focc['i']    
    count=0
    for i in range(0,len(f_occurrence_everyplanet)):
        if koi_table['koi_prad'][f_id[i]]>Rp_low and koi_table['koi_prad'][f_id[i]]<=Rp_high and koi_table['koi_period'][f_id[i]]>P_low and koi_table['koi_period'][f_id[i]]<=P_high:
#        and koi_table['koi_disposition'][f_id[i]]!='FALSE POSITIVE':
            count+=1
    
    if count>0:
        f_occurrence_temp=0
        for i in range(0,len(f_occurrence_everyplanet)):
            if koi_table['koi_prad'][f_id[i]]>Rp_low and koi_table['koi_prad'][f_id[i]]<=Rp_high and koi_table['koi_period'][f_id[i]]>P_low and koi_table['koi_period'][f_id[i]]<=P_high:
#            and koi_table['koi_disposition'][f_id[i]]!='FALSE POSITIVE':
#                N_s_temp=N_s(koi_table['koi_prad'][i],koi_table['koi_period'][i])
#                a_temp=a(koi_table['mass'][i],koi_table['koi_period'][i])
#                fgeo_temp=f_geo(koi_table['koi_prad'][i],koi_table['rad_b2018'][i],a_temp,e)
                if f_occurrence_everyplanet[i]!=np.inf:
                    f_occurrence_temp+=float(f_occurrence_everyplanet[i])
        
        temp=1.0*count/f_occurrence_temp
        if f_occurrence_temp>=1:
            return count,f_occurrence_temp,f_occurrence_temp/count**(0.5)
        else:
            deviation_N_s=temp*f_occurrence_temp*(1-f_occurrence_temp)
            std_N_s=deviation_N_s**0.5
            std_cell=std_N_s/temp
            return count,f_occurrence_temp,std_cell
    
    else:
        base_period=5
        base_radius=2
        Rp_center=base_radius**((math.log(Rp_low,base_radius)+math.log(Rp_high,base_radius))/2)
        P_center=base_period**((math.log(P_low,base_period)+math.log(P_high,base_period))/2)
        N_s_temp=N_s2(Rp_center,P_center)
        Rs_temp=1.0 #unit R_sun G type star as sun
        Ms_temp=1.0 #unit M_sun
        a_temp=a(Ms_temp,P_center)
        e=0.1
        fgeo_temp=f_geo(Rp_center,Rs_temp,a_temp,e)
        if N_s_temp==0:
            f_occurrence_temp=0
        else:
            f_occurrence_temp=1./(fgeo_temp*N_s_temp)
            
        return count,f_occurrence_temp,f_occurrence_temp


In [5]:
for i in range(0,len(koi_table)):
    N_s(koi_table['koi_prad'][i],koi_table['koi_period'][i],i)
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060


3500
3501
3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699


In [6]:
stellar_table=pd.read_csv('data/kepler_stellar_q17_add_dispersions_reduction_reduction_cool_2020.csv',sep=',')
koi_table=pd.read_csv('data/kepler_koi_q17_add_dispersions_reduction_reduction_2sigma_cool.csv',sep=',')

f_occurrence_everyplanet=[]
#os.system('cd l')
from subprocess import getoutput
filelist=getoutput('ls result/N_cool_2020/*.csv')
filename=[]
for line in filelist.split('\n'):
    filename.append(line)
    
print(filename)
#os.system('cd ..')
with open('result/occurrence_rate_cool_2020_2sigma.csv','w+') as f:
    s=['i',
       'kepid',
       'occurrence_rate',]
    writer=csv.DictWriter(f,fieldnames=s)
    writer.writeheader()
    for i in range(0,len(filename)):
        koi_id=int(filename[i][19:-4])
        P=koi_table['koi_period'][koi_id]
        Rp=koi_table['koi_prad'][koi_id]
        Ms=koi_table['mass'][koi_id]
        Rs=koi_table['rad_b2018'][koi_id]
        a_temp=a(Ms,P)
        N_inf=pd.read_csv(filename[i],sep=',')
        fn=N_inf['fn']
        feff=N_inf['feff']
        N_s_temp=0
        for j in range(0,len(fn)):
            N_s_temp+=fn[j]*feff[j]
        
        del N_inf,fn,feff
        gc.collect()
        N_s_temp=round(N_s_temp)
        fgeo_temp=f_geo(Rp,Rs,a_temp,e)
        f_occurrence_temp=1./(fgeo_temp*N_s_temp)
        f_occurrence_everyplanet.append(f_occurrence_temp)
        print(koi_id,f_occurrence_temp,koi_table['kepid'][koi_id],len(koi_table))
        writer.writerow({'i':koi_id,
                         'kepid':koi_table['kepid'][koi_id],
                         'occurrence_rate':f_occurrence_temp,})
 

['result/N_cool_2020/0.csv', 'result/N_cool_2020/1000.csv', 'result/N_cool_2020/1001.csv', 'result/N_cool_2020/1002.csv', 'result/N_cool_2020/1003.csv', 'result/N_cool_2020/1004.csv', 'result/N_cool_2020/1005.csv', 'result/N_cool_2020/1006.csv', 'result/N_cool_2020/1007.csv', 'result/N_cool_2020/1008.csv', 'result/N_cool_2020/1009.csv', 'result/N_cool_2020/100.csv', 'result/N_cool_2020/1010.csv', 'result/N_cool_2020/1011.csv', 'result/N_cool_2020/1012.csv', 'result/N_cool_2020/1013.csv', 'result/N_cool_2020/1014.csv', 'result/N_cool_2020/1015.csv', 'result/N_cool_2020/1016.csv', 'result/N_cool_2020/1017.csv', 'result/N_cool_2020/1018.csv', 'result/N_cool_2020/1019.csv', 'result/N_cool_2020/101.csv', 'result/N_cool_2020/1020.csv', 'result/N_cool_2020/1021.csv', 'result/N_cool_2020/1022.csv', 'result/N_cool_2020/1023.csv', 'result/N_cool_2020/1024.csv', 'result/N_cool_2020/1025.csv', 'result/N_cool_2020/1026.csv', 'result/N_cool_2020/1027.csv', 'result/N_cool_2020/1028.csv', 'result/N_co

0 0.0015559015808035833 8182107 4378
1000 0.00011226048597551039 9909735 4378
1001 0.0015223360986034687 6025174 4378
1002 0.00015519414175137097 9092504 4378
1003 4.0818258043558846e-05 9092504 4378
1004 5.802663764247248e-05 10294613 4378
1005 1.9661239622793545e-05 5887316 4378
1006 0.0008081020172581979 10843431 4378
1007 0.0002209670565802192 10843431 4378
1008 0.0002070234617053435 12356914 4378
1009 2.4197287347368073e-05 6706287 4378
100 0.0003701286823377257 11757422 4378
1010 2.8284962200883086e-05 2440757 4378
1011 0.0003997349371850606 11465651 4378
1012 3.0323747865893512e-05 5887393 4378
1013 3.201669197436862e-05 4074294 4378
1014 0.00043530862992102634 1026957 4378
1015 0.00034519454873207486 8547140 4378
1016 0.00022154617762150687 2832589 4378
1017 8.60848864804893e-05 2832589 4378
1018 0.00023342882149183746 12352520 4378
1019 0.0008418390909759056 10158729 4378
101 1.94323894000874e-05 6629588 4378
1020 3.302764482519382e-05 6228703 4378
1021 0.035825773763861236 11

1183 0.0006617992859084372 8868907 4378
1184 6.381305457072308e-05 9635510 4378
1185 0.001803277390194127 6230649 4378
1186 0.00026613883021032157 5257082 4378
1187 2.7557077947244322e-05 9635529 4378
1188 0.0007283832704256905 11295426 4378
1189 0.00010209882487200171 11295426 4378
118 0.0006376713263286386 10920281 4378
1190 9.47746916216354e-05 3231341 4378
1191 0.00018029400009820723 3231341 4378
1192 0.0001823029821525829 3231341 4378
1193 0.00013799415915932532 3231341 4378
1194 7.462910847469003e-06 10620329 4378
1195 0.0015271075810527218 10160922 4378
1196 0.0008996702456244664 2975770 4378
1197 0.003836620226271718 2975770 4378
1198 3.402429370817207e-05 10666230 4378
1199 0.0003829979401340967 7094944 4378
119 0.00011293782876500138 8013289 4378
11 6.093677481772357e-05 6628237 4378
1200 0.0009019544602260405 10005758 4378
1201 0.001579202514018501 10005758 4378
1202 0.0002694143416771699 4054784 4378
1203 0.00021992099884325124 9941387 4378
1204 4.405533378337831e-05 456811

1366 0.0009946925888057398 5385469 4378
1367 9.19534660910765e-05 4757437 4378
1368 0.00017016660677927344 4757437 4378
1369 0.00020310606912993652 5385471 4378
136 0.021633453038173645 8414216 4378
1370 0.00012487367283657986 8439323 4378
1371 0.0009980952803237283 9941859 4378
1372 0.00019407301016282114 9941859 4378
1373 0.000329115412054727 9941859 4378
1374 0.00022051182755852245 5511659 4378
1375 0.000386468480052723 5511659 4378
1376 0.00022941471193388557 5385491 4378
1377 0.00015467237627459887 8972058 4378
1378 8.050086146714993e-05 8972058 4378
1379 0.0022412581668807023 3934657 4378
137 0.00024987811591826287 9776794 4378
1380 0.00021164836773470883 10714072 4378
1381 0.00016545813346561958 7663691 4378
1382 0.0001705844368969702 9965957 4378
1383 0.0002664112551865652 5385667 4378
1384 0.00026607177145021203 5385697 4378
1385 0.00027302974714475355 5385697 4378
1386 2.2482840803230516e-05 9300285 4378
1387 0.00014362291191073815 4459068 4378
1388 0.0003227947319276162 8331

1549 9.846768357234119e-05 10388286 4378
154 0.0008807408557746762 10600955 4378
1550 0.0003532604274769432 11045383 4378
1551 2.6558520468437936e-05 7115463 4378
1552 9.636971712610527e-05 7831264 4378
1553 0.00017366266528328668 7831264 4378
1554 0.0001328156597418703 3838486 4378
1555 3.1507075778414524e-05 6847018 4378
1556 0.0006040787444312052 7455287 4378
1557 0.0002788603195828633 7455287 4378
1558 0.0011327922895409991 7455287 4378
1559 0.0017552326886537445 3958301 4378
155 0.0003132105713533271 6605493 4378
1560 0.00017224246220416567 3439126 4378
1561 0.00045116012841151486 6864885 4378
1562 0.00019910200152369205 5959753 4378
1563 0.01118184981228264 6864891 4378
1564 8.387869007274732e-05 7902204 4378
1565 0.0011298747454480723 10003058 4378
1566 0.001212123647209039 6864893 4378
1567 4.388242226563084e-05 7455553 4378
1568 0.00011757203603071818 9266285 4378
1569 0.00041281477936571645 8973129 4378
156 0.0006478242629242663 4633570 4378
1570 0.0005677227931461488 9782691

...

4163 0.0003080567141731382 6936977 4378
4164 0.007711472304158187 8560927 4378
4165 0.0005602528930138456 9409493 4378
4166 3.4349398543455995e-05 4949524 4378
4167 0.0020439199473923936 8560940 4378
4168 0.0005054026179670679 5443320 4378
4169 0.0002464307407141909 11026304 4378
416 0.0003314547636284447 10252275 4378
4170 0.00018765554917113948 6290935 4378
4171 0.0003946018512515532 8628665 4378
4172 0.0014299771597513933 8813698 4378
4173 0.00015425794529782744 10747501 4378
4174 7.105829035850897e-05 9790806 4378
4175 0.00037539052988550675 10122255 4378
4176 0.0006983131020001905 11506654 4378
4177 0.0002300491893029276 4949751 4378
4178 2.910088129723302e-05 9474483 4378
4179 0.0021985598258700166 9166700 4378
417 0.0006098488478080815 7778427 4378
4180 0.00054464674196217 6603043 4378
4181 3.0459796212201948e-05 9474493 4378
4182 0.0002623769394980278 9836149 4378
4183 2.370510463983546e-05 10202759 4378
4184 0.000160764315223674 10907132 4378
4185 4.1437212575579253e-05 840958

4346 0.00019648650846044145 5687700 4378
4347 7.304217521677197e-05 6543645 4378
4348 7.430337498081991e-05 2696205 4378
4349 0.00014174172371379107 8364115 4378
434 4.934273063531229e-05 9899449 4378
4350 4.837723305724498e-05 3119256 4378
4351 7.300347420411785e-05 4055765 4378
4352 4.414132489064055e-05 3964545 4378
4353 0.0006877771519138688 7024530 4378
4354 0.004560827595667618 8364232 4378
4355 0.0001230605704206972 6185717 4378
4356 4.967030649741657e-05 3964562 4378
4357 1.5774961708299044e-05 8521059 4378
4358 0.0002648087019320775 9019191 4378
4359 0.004486760862895687 11558217 4378
435 0.00011499725330865073 11176166 4378
4360 0.000491677601552677 11507101 4378
4361 9.74864023443699e-05 5596440 4378
4362 6.810435342975975e-05 11073351 4378
4363 8.647812123884994e-05 11072016 4378
4364 0.0016287102681818706 10717241 4378
4365 0.0002943698704388068 10717241 4378
4366 0.00038234074716000764 9540675 4378
4367 0.0009369761493518745 8212002 4378
4368 0.002570710455486059 8212002 

593 5.509287405414909e-05 2989404 4378
594 0.003873400184263839 10079481 4378
595 0.01039280101327944 4472143 4378
596 0.00026306458478631353 4857213 4378
597 0.00013043085366231026 8492101 4378
598 9.780470673729767e-05 5357901 4378
599 0.0003222725448132214 7916058 4378
59 0.00012829485410592662 5211199 4378
5 8.426703097075532e-05 11037128 4378
600 5.3211178296295685e-05 11463950 4378
601 0.00014089828427568806 11457726 4378
602 0.0006980948400773066 9594184 4378
603 0.00017250109269613729 7501646 4378
604 3.670096186114461e-05 7032568 4378
605 0.000301961605383741 9570741 4378
606 3.0652691196306764e-05 7032687 4378
607 0.00022248856679966435 11656246 4378
608 3.8095278721815525e-05 5802292 4378
609 4.239568922155565e-05 7770450 4378
60 0.00017743851052021972 5211199 4378
610 1.826107515296868e-05 7770471 4378
611 0.00034140512653530895 9405595 4378
612 8.71732551158308e-05 3757407 4378
613 0.00011129032878954 7289165 4378
614 0.0003997587304068406 11153476 4378
615 4.1925795965012

781 0.00015199899774009088 2837111 4378
782 2.9452853747306712e-05 9541163 4378
783 3.56063081609569e-05 7211879 4378
784 0.00015044793621971577 11402995 4378
785 0.0013917871946686445 4680772 4378
786 0.000191353418942112 6029239 4378
787 0.0129647395947793 6029239 4378
788 0.00014983770717023407 9934208 4378
789 9.025221733649747e-05 10318874 4378
78 1.801547584554368e-05 11774303 4378
790 3.924880733445546e-05 8189801 4378
791 0.00016029192158078937 6345732 4378
792 0.0002378700096916475 6345732 4378
793 4.783067735198215e-05 6595662 4378
794 0.00011725604530747728 9934274 4378
795 0.0001545491401992387 11125613 4378
796 0.0003014111354201115 11125613 4378
797 0.0002237552317972635 11125613 4378
798 7.924581670355186e-05 11403044 4378
799 7.131425840498876e-05 7025846 4378
79 0.0016562203092036893 9455677 4378
7 0.0029449128554470444 4277304 4378
800 0.0005021350978687231 10552611 4378
801 0.0002259634336242239 10552611 4378
802 0.00018901593654669838 4056616 4378
803 3.590885458136

96 0.00021730891465185476 6359802 4378
970 0.0005679405099604795 6511182 4378
971 0.0003079251095651003 4180401 4378
972 0.00015130456150015895 2708343 4378
973 4.59830761631415e-05 4073730 4378
974 0.00684283644557724 1025986 4378
975 0.000331702015149503 9632895 4378
976 0.004518552226716551 6182508 4378
977 0.00011881372117407834 5807579 4378
978 0.0026264945382124816 11125797 4378
979 0.0008227744224149746 11125797 4378
97 3.3960602209795594e-05 11774387 4378
980 7.028402851786111e-05 10875245 4378
981 0.00029654718768391944 10875245 4378
982 9.280396680297569e-05 10875245 4378
983 0.0001760158457270725 10875245 4378
984 0.00022300168548200592 9634821 4378
985 0.0009129716251599872 9634821 4378
986 0.00016376773420093154 9634821 4378
987 0.0002714615129539396 10271806 4378
988 0.00013205944044108445 10271806 4378
989 0.00016416477900682677 10271806 4378
98 3.071380735595754e-05 4815612 4378
990 0.000409792832782075 10271806 4378
991 0.002270506223924586 6863158 4378
992 0.000195104

In [3]:
bins_period=np.logspace(math.log(0.4,5),math.log(250,5),13,base=5)
bins_radius=np.logspace(-1.5,5.5,15,base=2)   

occurrence_cell=np.zeros((14,12))
std_cell=np.zeros((14,12))
count_cell=np.zeros((14,12))
for i in range(0,len(bins_radius)-1):
    for j in range(0,len(bins_period)-1):
        Rp_low=bins_radius[i]
        Rp_high=bins_radius[i+1]
        P_low=bins_period[j]
        P_high=bins_period[j+1]
        temp=f_occurrence_cell_nofp(Rp_low,Rp_high,P_low,P_high)
        count_cell[i,j]=temp[0]
        occurrence_cell[i,j]=temp[1]
        std_cell[i,j]=temp[2]
        print(i,j)

soften_factor=1./np.log10(bins_period[1]/bins_period[0])/np.log10(bins_radius[1]/bins_radius[0])
print(soften_factor)
with open('result/occurrence_rate_2sigma_cool_cell_nofp.csv','w') as f:
    s=['i',
       'j',
       'count_cell',
       'occurrence_cell',
       'std_cell',]
    writer=csv.DictWriter(f,fieldnames=s)
    writer.writeheader()    
    for i in range(0,len(bins_radius)-1):
        for j in range(0,len(bins_period)-1):
            writer.writerow({'i':i,
                             'j':j,
                             'count_cell':count_cell[i,j],
                             'occurrence_cell':occurrence_cell[i,j]*soften_factor,
                             'std_cell':std_cell[i,j]*soften_factor})

0 0
0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
0 10
0 11
1 0
1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
1 10
1 11
2 0
2 1
2 2
2 3
2 4
2 5
2 6
2 7
2 8
2 9
2 10
2 11
3 0
3 1
3 2
3 3
3 4
3 5
3 6
3 7
3 8
3 9
3 10
3 11
4 0
4 1
4 2
4 3
4 4
4 5
4 6
4 7
4 8
4 9
4 10
4 11
5 0
5 1
5 2
5 3
5 4
5 5
5 6
5 7
5 8
5 9
5 10
5 11
6 0
6 1
6 2
6 3
6 4
6 5
6 6
6 7
6 8
6 9
6 10
6 11
7 0
7 1
7 2
7 3
7 4
7 5
7 6
7 7
7 8
7 9
7 10
7 11
8 0
8 1
8 2
8 3
8 4
8 5
8 6
8 7
8 8
8 9
8 10
8 11
9 0
9 1
9 2
9 3
9 4
9 5
9 6
9 7
9 8
9 9
9 10
9 11
10 0
10 1
10 2
10 3
10 4
10 5
10 6
10 7
10 8
10 9
10 10
10 11
11 0
11 1
11 2
11 3
11 4
11 5
11 6
11 7
11 8
11 9
11 10
11 11
12 0
12 1
12 2
12 3
12 4
12 5
12 6
12 7
12 8
12 9
12 10
12 11
13 0
13 1
13 2
13 3
13 4
13 5
13 6
13 7
13 8
13 9
13 10
13 11
28.515627917764547


In [6]:
bins_period=np.logspace(math.log(0.4,5),math.log(250,5),13,base=5)
bins_radius=np.logspace(-1.5,5.5,15,base=2)   

occurrence_cell=np.zeros((14,12))
std_cell=np.zeros((14,12))
count_cell=np.zeros((14,12))
for i in range(0,len(bins_radius)-1):
    for j in range(0,len(bins_period)-1):
        Rp_low=bins_radius[i]
        Rp_high=bins_radius[i+1]
        P_low=bins_period[j]
        P_high=bins_period[j+1]
        temp=f_occurrence_cell_fp(Rp_low,Rp_high,P_low,P_high)
        count_cell[i,j]=temp[0]
        occurrence_cell[i,j]=temp[1]
        std_cell[i,j]=temp[2]
        print(i,j)

soften_factor=1./np.log10(bins_period[1]/bins_period[0])/np.log10(bins_radius[1]/bins_radius[0])
print(soften_factor)
with open('result/occurrence_rate_2sigma_cool_cell_fp.csv','w') as f:
    s=['i',
       'j',
       'count_cell',
       'occurrence_cell',
       'std_cell',]
    writer=csv.DictWriter(f,fieldnames=s)
    writer.writeheader()    
    for i in range(0,len(bins_radius)-1):
        for j in range(0,len(bins_period)-1):
            writer.writerow({'i':i,
                             'j':j,
                             'count_cell':count_cell[i,j],
                             'occurrence_cell':occurrence_cell[i,j]*soften_factor,
                             'std_cell':std_cell[i,j]*soften_factor})

0 0
0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
0 10
0 11
1 0
1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
1 10
1 11
2 0
2 1
2 2
2 3
2 4
2 5
2 6
2 7
2 8
2 9
2 10
2 11
3 0
3 1
3 2
3 3
3 4
3 5
3 6
3 7
3 8
3 9
3 10
3 11
4 0
4 1
4 2
4 3
4 4
4 5
4 6
4 7
4 8
4 9
4 10
4 11
5 0
5 1
5 2
5 3
5 4
5 5
5 6
5 7
5 8
5 9
5 10
5 11
6 0
6 1
6 2
6 3
6 4
6 5
6 6
6 7
6 8
6 9
6 10
6 11
7 0
7 1
7 2
7 3
7 4
7 5
7 6
7 7
7 8
7 9
7 10
7 11
8 0
8 1
8 2
8 3
8 4
8 5
8 6
8 7
8 8
8 9
8 10
8 11
9 0
9 1
9 2
9 3
9 4
9 5
9 6
9 7
9 8
9 9
9 10
9 11
10 0
10 1
10 2
10 3
10 4
10 5
10 6
10 7
10 8
10 9
10 10
10 11
11 0
11 1
11 2
11 3
11 4
11 5
11 6
11 7
11 8
11 9
11 10
11 11
12 0
12 1
12 2
12 3
12 4
12 5
12 6
12 7
12 8
12 9
12 10
12 11
13 0
13 1
13 2
13 3
13 4
13 5
13 6
13 7
13 8
13 9
13 10
13 11
28.515627917764547


In [3]:
stellar_table=pd.read_csv('data/kepler_stellar_q17_add_dispersions_reduction_reduction_hot_2020.csv',sep=',')
koi_table=pd.read_csv('data/kepler_koi_q17_add_dispersions_reduction_reduction_2sigma_hot.csv',sep=',')

f_occurrence_everyplanet=[]
#os.system('cd l')
from subprocess import getoutput
filelist=getoutput('ls result/N_hot_2020/*.csv')
filename=[]
for line in filelist.split('\n'):
    filename.append(line)
    
print(filename)
#os.system('cd ..')
with open('result/occurrence_rate_hot_2020_2sigma.csv','w+') as f:
    s=['i',
       'kepid',
       'occurrence_rate',]
    writer=csv.DictWriter(f,fieldnames=s)
    writer.writeheader()
    for i in range(0,len(filename)):
        koi_id=int(filename[i][18:-4])
        P=koi_table['koi_period'][koi_id]
        Rp=koi_table['koi_prad'][koi_id]
        Ms=koi_table['mass'][koi_id]
        Rs=koi_table['rad_b2018'][koi_id]
        a_temp=a(Ms,P)
        N_inf=pd.read_csv(filename[i],sep=',')
        fn=N_inf['fn']
        feff=N_inf['feff']
        N_s_temp=0
        for j in range(0,len(fn)):
            N_s_temp+=fn[j]*feff[j]
        
        del N_inf,fn,feff
        gc.collect()
        N_s_temp=round(N_s_temp)
        fgeo_temp=f_geo(Rp,Rs,a_temp,e)
        f_occurrence_temp=1./(fgeo_temp*N_s_temp)
        f_occurrence_everyplanet.append(f_occurrence_temp)
        print(koi_id,f_occurrence_temp,koi_table['kepid'][koi_id],len(koi_table))
        writer.writerow({'i':koi_id,
                         'kepid':koi_table['kepid'][koi_id],
                         'occurrence_rate':f_occurrence_temp,})
 

['result/N_hot_2020/0.csv', 'result/N_hot_2020/100.csv', 'result/N_hot_2020/101.csv', 'result/N_hot_2020/102.csv', 'result/N_hot_2020/103.csv', 'result/N_hot_2020/104.csv', 'result/N_hot_2020/105.csv', 'result/N_hot_2020/106.csv', 'result/N_hot_2020/107.csv', 'result/N_hot_2020/108.csv', 'result/N_hot_2020/109.csv', 'result/N_hot_2020/10.csv', 'result/N_hot_2020/110.csv', 'result/N_hot_2020/111.csv', 'result/N_hot_2020/112.csv', 'result/N_hot_2020/113.csv', 'result/N_hot_2020/114.csv', 'result/N_hot_2020/115.csv', 'result/N_hot_2020/116.csv', 'result/N_hot_2020/117.csv', 'result/N_hot_2020/118.csv', 'result/N_hot_2020/119.csv', 'result/N_hot_2020/11.csv', 'result/N_hot_2020/120.csv', 'result/N_hot_2020/121.csv', 'result/N_hot_2020/122.csv', 'result/N_hot_2020/123.csv', 'result/N_hot_2020/124.csv', 'result/N_hot_2020/125.csv', 'result/N_hot_2020/126.csv', 'result/N_hot_2020/127.csv', 'result/N_hot_2020/128.csv', 'result/N_hot_2020/129.csv', 'result/N_hot_2020/12.csv', 'result/N_hot_2020

0 0.00036921904456304656 5120225 335
100 0.0008470625298067345 11192235 335
101 0.0024205083552872354 9049550 335
102 0.0003539059654068272 8260198 335
103 0.00023614785754849307 7137798 335
104 0.004020317869163502 10873178 335
105 0.0034113451352201147 4645174 335
106 0.0021002597549214827 4645174 335
107 0.0004904279251716041 8526387 335
108 0.013307310852928642 5959719 335
109 0.02399435765816257 5959719 335
10 0.001866528327578874 3241557 335
110 0.00032601421882448285 6929016 335
111 0.001172045654474065 9468717 335
112 0.015138108309321585 9782748 335
113 0.006730295704518538 6364143 335
114 0.00033970892577791086 5428471 335
115 0.0016605088609617953 12068975 335
116 0.003516681977703725 12068975 335
117 0.02217872153954472 12068975 335
118 0.002321748639683742 12068975 335
119 0.012014873342705774 8017703 335
11 0.0020003631364627376 8042453 335
120 0.033355466325398764 8017703 335
121 0.0031375047567846117 8017703 335
122 0.0012887781323169768 9773980 335
123 0.00313905918162

297 0.0011478488839303305 7620660 335
298 0.007222889598535513 10904004 335
299 0.002417481692999361 4948730 335
29 0.0030409130646746577 11499263 335
2 0.006117547639227289 10600261 335
300 0.0015374276330847624 6781548 335
301 0.0011722291205855417 9520668 335
302 0.04413687049438618 6685403 335
303 0.001813875895287213 2161949 335
304 0.0021751287953887086 2161949 335
305 0.011572228673421897 5015459 335
306 0.005844811056633244 5632701 335
307 0.0005040543034980553 6268722 335
308 0.00033815793324930694 11125936 335
309 0.000972428248391398 3526331 335
30 0.0002499287196186915 9153554 335
310 0.0020803167147650323 7668648 335
311 0.0025996066095483835 9334490 335
312 0.002111154569791965 4263529 335
313 0.00449400617820185 5796186 335
314 0.0002585904990347593 5040223 335
315 0.021719351527679738 5860347 335
316 0.0010784093214596328 8561063 335
317 0.0028422379674135564 8561063 335
318 0.008140183803414462 8561063 335
319 0.016568364690514557 10907307 335
31 0.0005591362577214036 

In [2]:
data=pd.read_csv("result/occurrence_rate_2sigma_cool_cell_fp.csv")

with open('result/occurrence_rate_2sigma_cool_cell_fp_new.csv','w') as f:
    s=['0','1','2','3','4','5','6','7','8','9','10','11']
    writer=csv.DictWriter(f,fieldnames=s)
    writer.writeheader()    
    for i in range(27,-1,-1):
        temp=[]
        for j in range(0,12):         
            if np.mod(i,2)!=0:
                if data['count_cell'][int(i/2)*12+j]==0:
                    temp.append('')
                    
                else:
                    temp.append('{:.2g}'.format(data['occurrence_cell'][int(i/2)*12+j]))
                    
            
            elif np.mod(i,2)==0:
                if data['count_cell'][int(i/2)*12+j]==0:
                    temp.append('<'+'{:.2g}'.format(data['std_cell'][int(i/2)*12+j]))   
                                                 
                else:
                    temp.append('$\pm$'+'{:.2g}'.format(data['std_cell'][int(i/2)*12+j])) 
                    
                                        
        writer.writerow({'0':temp[0],
                         '1':temp[1],
                         '2':temp[2],
                         '3':temp[3],
                         '4':temp[4],
                         '5':temp[5],
                         '6':temp[6],
                         '7':temp[7],
                         '8':temp[8],
                         '9':temp[9],
                         '10':temp[10],
                         '11':temp[11],})
        
data=pd.read_csv("result/occurrence_rate_2sigma_cool_cell_fp_new.csv")
data

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.003,0.015,0.011,0.021,0.042,0.042,0.067,0.084,0.12,0.1,0.12,0.27
1,$\pm$0.0012,$\pm$0.0032,$\pm$0.0031,$\pm$0.005,$\pm$0.0092,$\pm$0.011,$\pm$0.017,$\pm$0.022,$\pm$0.033,$\pm$0.034,$\pm$0.046,$\pm$0.085
2,0.0097,0.017,0.029,0.039,0.05,0.05,0.072,0.11,0.086,0.094,0.041,0.026
3,$\pm$0.0024,$\pm$0.0036,$\pm$0.0056,$\pm$0.0081,$\pm$0.011,$\pm$0.012,$\pm$0.018,$\pm$0.028,$\pm$0.03,$\pm$0.035,$\pm$0.029,$\pm$0.026
4,0.0081,0.017,0.0065,0.024,0.028,0.06,0.065,0.062,0.068,0.021,0.016,0.03
5,$\pm$0.0024,$\pm$0.0039,$\pm$0.0029,$\pm$0.0065,$\pm$0.0076,$\pm$0.015,$\pm$0.018,$\pm$0.022,$\pm$0.028,$\pm$0.021,$\pm$0.016,$\pm$0.03
6,0.0026,0.0068,0.0094,0.034,0.026,0.024,0.027,0.049,0.07,0.08,0.14,0.033
7,$\pm$0.0013,$\pm$0.0026,$\pm$0.0033,$\pm$0.0081,$\pm$0.0078,$\pm$0.0086,$\pm$0.011,$\pm$0.017,$\pm$0.028,$\pm$0.036,$\pm$0.063,$\pm$0.033
8,0.0015,0.0039,0.0076,0.022,0.017,0.024,0.057,0.017,0.097,0.11,0.34,0.23
9,$\pm$0.00089,$\pm$0.0019,$\pm$0.0031,$\pm$0.0063,$\pm$0.0065,$\pm$0.009,$\pm$0.018,$\pm$0.012,$\pm$0.034,$\pm$0.044,$\pm$0.093,$\pm$0.092


In [4]:
data=pd.read_csv("result/occurrence_rate_2sigma_cool_cell_nofp.csv")

with open('result/occurrence_rate_2sigma_cool_cell_nofp_new.csv','w') as f:
    s=['0','1','2','3','4','5','6','7','8','9','10','11']
    writer=csv.DictWriter(f,fieldnames=s)
    writer.writeheader()    
    for i in range(27,-1,-1):
        temp=[]
        for j in range(0,12):         
            if np.mod(i,2)!=0:
                if data['count_cell'][int(i/2)*12+j]==0:
                    temp.append('')
                    
                else:
                    temp.append('{:.2g}'.format(data['occurrence_cell'][int(i/2)*12+j]))
                    
            
            elif np.mod(i,2)==0:
                if data['count_cell'][int(i/2)*12+j]==0:
                    temp.append('<'+'{:.2g}'.format(data['std_cell'][int(i/2)*12+j]))   
                                                 
                else:
                    temp.append('$\pm$'+'{:.2g}'.format(data['std_cell'][int(i/2)*12+j])) 
                    
                                        
        writer.writerow({'0':temp[0],
                         '1':temp[1],
                         '2':temp[2],
                         '3':temp[3],
                         '4':temp[4],
                         '5':temp[5],
                         '6':temp[6],
                         '7':temp[7],
                         '8':temp[8],
                         '9':temp[9],
                         '10':temp[10],
                         '11':temp[11],})
        
data=pd.read_csv("result/occurrence_rate_2sigma_cool_cell_nofp_new.csv")
data

,0,1,2,3,4,5,6,7,8,9,10,11
0,NaN,0.0014,0.00091,NaN,NaN,NaN,NaN,0.012,0.012,NaN,NaN,0.028
1,<0.00055,$\pm$0.00099,$\pm$0.00091,<0.0016,<0.0023,<0.0033,<0.0047,$\pm$0.0082,$\pm$0.012,<0.014,<0.021,$\pm$0.028
2,0.00083,0.00056,0.0027,0.005,0.0018,0.0013,0.02,0.013,NaN,NaN,NaN,NaN
3,$\pm$0.00083,$\pm$0.00056,$\pm$0.0016,$\pm$0.0029,$\pm$0.0018,$\pm$0.0013,$\pm$0.01,$\pm$0.0089,<0.011,<0.015,<0.023,<0.033
4,NaN,0.001,0.0012,0.0017,0.0084,0.015,0.018,0.014,NaN,NaN,NaN,0.03
5,<0.00063,$\pm$0.001,$\pm$0.0012,$\pm$0.0017,$\pm$0.0042,$\pm$0.0073,$\pm$0.011,$\pm$0.0096,<0.011,<0.016,<0.024,$\pm$0.03
6,NaN,NaN,0.0035,0.019,0.016,0.0095,0.0037,0.019,0.036,0.045,0.057,0.033
7,<0.00066,<0.00094,$\pm$0.002,$\pm$0.006,$\pm$0.0065,$\pm$0.0055,$\pm$0.0037,$\pm$0.011,$\pm$0.021,$\pm$0.026,$\pm$0.04,$\pm$0.033
8,NaN,NaN,0.0042,0.0083,0.017,0.015,0.022,0.0064,0.069,0.073,0.29,0.18
9,<0.00068,<0.00098,$\pm$0.0024,$\pm$0.0042,$\pm$0.0065,$\pm$0.0075,$\pm$0.011,$\pm$0.0064,$\pm$0.028,$\pm$0.036,$\pm$0.087,$\pm$0.082
